# Enhasing the Data with Census FIPS and Ecosystem Data
### Purpose
In this notebook I will add in columns to the working data set that will contain 1) The census declaired blocks FIPS and County FIPS and 2) USGS declared ecosystems for each CBC location and 3) USGS declared ecosystems for each NOAA station location.

The census declaired blocks FIPS and County FIPS codes are the unquie identifyers census uses to identify an area. To learn more you can visit: https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html


### Author: 
Ren C'deBaca
### Date: 
2020-04-21
### Update Date: 
2020-04-26

### Inputs 
1.0-rec-initial-data-cleaning.txt - Tab seperated file of cleaned Christmas Bird Count events  Each row represents a single count in a given year. Data Dictonary can be found here: http://www.audubon.org/sites/default/files/documents/cbc_report_field_definitions_2013.pdf

np-circles-to-ecosys_data.csv - Comma seperated file from Nathan Pavlovic(nathan.pavlovic@gmail.com). This file was produced by first passing Nathan a file of approximatly 4000 unique lat lon present in the clean data file. 

Nathan then used the 2008 USGS raster ecosystem dataset. Info here https://rmgsc.cr.usgs.gov/outgoing/ecosystems/USdata/  

He used the Extract Values to Points tool in ArcGIS to find the raster value at each point. 

unique_stations_latlong_ecosys.csv _ Comma seperated file from Nathan Pavlovic(nathan.pavlovic@gmail.com). This file was produced by first passing Nathan a file of the unique NOAA station lat lons that were present in the file 1.1-circles_to_many_stations_usa_weather_data_20200424213015.csv.  See the abouve notes on his process.

1.2-ijd-fetch-circle-elevations_20200502155633.csv - CSV file of cbc circles matched with NOAA stations and elivation data. Each row is a cbc circle matched to a NOAA station. A cbc location can appear on multiple rows if they are matched to multiple stations




### Output Files
1.3-rec-connecting-fips-data.csv -- CSV file of the unique lat lons present in cbc data. Each Lat lon is matched to a Block FIPS and County FIPS. (This is the file that was shared with Nathan) 

1.3-rec-connecting-fips-ecosystem-data -- CSV file of the station matched cbc data with added columns for Ecosystem data for cbc circles and NOAA stations and cencus FIPS data


## Steps or Proceedures in the notebook 
1. Load in the cleaned data 
2. Identify the unique Lat Lons present in the cbc circle locations 
3. 
    OPTION 1: Load in the saved census FIPS data
    OPTION 2: Run the data through the census API (Note: Takes a few hours) 
5. Load in Ecological Data from Nathan
6. Create a key to based on the lat long of the cbc circles to merge the station matched data with the ecological data
7. Merge in the census FIPS data, the cbc ecological data, and the noaa station ecological data 


## Where the Data will Be Saved 
The raw ecosystem data and the output data will be saved in the Google Drive Folder
https://drive.google.com/drive/folders/1Nlj9Nq-_dPFTDbrSDf94XMritWYG6E2I

The path should look like this: 
audubon-cbc/data/Cloud_Data/<DATA FILE>

## Reference
    https://geo.fcc.gov/api/census/#!/block/get_block_find


In [202]:
# Imports
import os
from datetime import datetime
# Version .24.0
from google.cloud import bigquery
import pandas as pd
import pandas
import requests
import time
import numpy as np

pd.set_option('display.max_columns', 500)
pd.options.display.max_rows = 999

In [203]:
# ALL File Paths should be declared at the TOP of the notebook
PATH_TO_CLEAN_CBC_DATA = "../data/Cloud_Data/1.0-rec-initial-data-cleaning.txt"
PATH_TO_WORKING_DATA = "../data/Cloud_Data/1.2.1-ijd-fetch-circle-elevations_20200709104212.txt"


PATH_TO_CBC_ECO_DATA = "../data/np-circles-to-ecosys_data.csv" 
PATH_TO_NOAA_ECO_DATA = "../data/unique_stations_latlong_ecosys.csv"



USE_CENSUS_BACKUP_FILE = True

## Load in the Clean Data

In [204]:
clean_data = pd.read_csv(PATH_TO_CLEAN_CBC_DATA, encoding = "ISO-8859-1", sep="\t")

/Users/rcdebaca/.pyenv/versions/funhacks371/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (29,30) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [205]:
clean_data.shape

(90411, 48)

In [206]:
clean_data['ui'].nunique()

90411

In [207]:
clean_data.head()

,circle_name,country_state,lat,lon,count_year,count_date,n_field_counters,n_feeder_counters,min_field_parties,max_field_parties,field_hours,feeder_hours,nocturnal_hours,field_distance,nocturnal_distance,distance_units,min_temp,max_temp,temp_unit,min_wind,max_wind,wind_unit,min_snow,max_snow,snow_unit,am_cloud,pm_cloud,am_rain,pm_rain,am_snow,pm_snow,field_distance_imperial,field_distance_metric,nocturnal_distance_imperial,nocturnal_distance_metric,min_snow_imperial,min_snow_metric,max_snow_metric,max_snow_imperial,min_temp_imperial,max_temp_imperial,min_temp_metric,max_temp_metric,min_wind_metric,max_wind_metric,min_wind_imperial,max_wind_imperial,ui
0,Pacific Grove,US-CA,36.616700,-121.916700,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.6167-121.9167_1901
1,Pueblo,US-CO,38.175251,-104.519575,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.175251-104.519575_1901
2,Bristol,US-CT,41.671800,-72.949500,1901,1900-12-25,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.6718-72.9495_1901
3,Norwalk,US-CT,41.116700,-73.400000,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.1167-73.4_1901
4,Glen Ellyn,US-IL,41.883300,-88.066700,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.8833-88.0667_1901


### Create a string key to represent a unique lat lon combonation 

In [208]:
clean_data['temp_key_str'] = clean_data['lat'].astype(str) + clean_data['lon'].astype(str)

In [209]:
clean_data['temp_key_str'].nunique()

4585

In [210]:
clean_data.head(50)

,circle_name,country_state,lat,lon,count_year,count_date,n_field_counters,n_feeder_counters,min_field_parties,max_field_parties,field_hours,feeder_hours,nocturnal_hours,field_distance,nocturnal_distance,distance_units,min_temp,max_temp,temp_unit,min_wind,max_wind,wind_unit,min_snow,max_snow,snow_unit,am_cloud,pm_cloud,am_rain,pm_rain,am_snow,pm_snow,field_distance_imperial,field_distance_metric,nocturnal_distance_imperial,nocturnal_distance_metric,min_snow_imperial,min_snow_metric,max_snow_metric,max_snow_imperial,min_temp_imperial,max_temp_imperial,min_temp_metric,max_temp_metric,min_wind_metric,max_wind_metric,min_wind_imperial,max_wind_imperial,ui,temp_key_str
0,Pacific Grove,US-CA,36.616700,-121.916700,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.6167-121.9167_1901,36.6167-121.9167
1,Pueblo,US-CO,38.175251,-104.519575,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.175251-104.519575_1901,38.175251-104.519575
2,Bristol,US-CT,41.671800,-72.949500,1901,1900-12-25,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.6718-72.9495_1901,41.6718-72.9495
3,Norwalk,US-CT,41.116700,-73.400000,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.1167-73.4_1901,41.1167-73.4
4,Glen Ellyn,US-IL,41.883300,-88.066700,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.8833-88.0667_1901,41.8833-88.0667
5,Baldwin,US-LA,29.833300,-91.550000,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.8333-91.55_1901,29.8333-91.55
6,Belmont,US-MA,42.383300,-71.166700,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.3833-71.1667_1901,42.3833-71.1667
7,Greater Boston,US-MA,42.397309,-71.095501,1901,1900-12-26,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.397309-71.095501_1901,42.397309-71.095501
8,Winchester,US-MA,42.450000,-71.133300,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.45-71.1333_1901,42.45-71.1333
9,La Grange,US-MO,40.050000,-91.500000,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.05-91.5_1901,40.05-91.5


In [211]:
clean_data_for_nathan = clean_data[["lat", "lon", "ui", "temp_key_str"]]
clean_data_for_nathan.shape

(90411, 4)

In [212]:
clean_data_for_nathan = clean_data_for_nathan.drop_duplicates("temp_key_str")
clean_data_for_nathan.shape

(4585, 4)

In [213]:
clean_data_for_nathan = clean_data_for_nathan[["lon", "lat"]]

In [214]:
clean_data_for_nathan.to_csv('../data/Cloud_data/1.3-rec-clean_data_for_gis.csv', index=False)



## Census Data 
There are two options here
OPTION 1: Send the unique lat lons though a census API to find the block and county fips 
OPTION 2: Load in the saved census FIPS data

### Option 1: Load in the saved census FIPS data 

In [215]:
## Option: Set USE_CENSUS_BACKUP_FILE to True to use the file from backup
if USE_CENSUS_BACKUP_FILE:
    census_prep_df = pd.read_csv("1.3-rec-connecting-fips-data.csv")
    census_prep_df = census_prep_df[["lat", "lon", "block_fips", "county_fips"]]
    census_prep_df['temp_key_str'] = census_prep_df['lat'].astype(str) + census_prep_df['lon'].astype(str)
    print(clean_data.shape)
    census_prep_df.head()

(90411, 49)


### Option 2: Run the data through the census API (Note: Takes a few hours) 

In [216]:
if not USE_CENSUS_BACKUP_FILE:
    # Create a small dataframe of unique lat lon location to use with cencus data 
    census_prep_df = clean_data[['temp_key_str', 'lat', 'lon']]

In [217]:
if not USE_CENSUS_BACKUP_FILE:
    census_prep_df.shape

In [218]:
if not USE_CENSUS_BACKUP_FILE:
    # Drop duplicate rows 
    census_prep_df = census_prep_df.drop_duplicates(subset=['lat', 'lon'], keep= 'first') 
    print(census_prep_df.shape)

### Create a test call to the API to see how the data comes back 

In [219]:
if not USE_CENSUS_BACKUP_FILE:
    # Test Lat and Lon
    lat = 51.409713
    lon = 179.284881

    BASE_URL = "https://geo.fcc.gov/api/census/block/find?format=json&latitude=%s&longitude=%s"
    url = BASE_URL % (lat, lon)

    payload = {}
    headers= {}

    response = requests.request("GET", url, headers=headers, data = payload)

    print(response.text.encode('utf8'))

### Build a loop to build of list of results from the census API to get the block FIPS code and county FIPS code

In [220]:
if not USE_CENSUS_BACKUP_FILE:
    result_list = []
    county_result_list = []

    BASE_URL = "https://geo.fcc.gov/api/census/block/find?format=json&latitude=%s&longitude=%s"

    TIME_DELAY = 2

    for index, row in census_prep_df.iterrows():
        block_fips = ''
        county_fips = ''

        lat = row['lat']
        lon = row['lon']

        url = BASE_URL % (lat, lon)
        payload = {}
        headers= {}
        response = requests.request("GET", url, headers=headers, data = payload)

        try:
            block_fips = response.json()['Block']['FIPS']
            county_fips = response.json()['County']['FIPS']
        except:
            "Could not get FIPS "

        result_list.append(block_fips)
        county_result_list.append(county_fips)

        time.sleep(TIME_DELAY)


In [221]:
if not USE_CENSUS_BACKUP_FILE:
    print(len(result_list))
    print(len(county_result_list))


In [222]:
if not USE_CENSUS_BACKUP_FILE:
    # Turn the result list into arrays 
    result_arry = pd.Series(result_list)
    county_array = pd.Series(county_result_list)

In [223]:
if not USE_CENSUS_BACKUP_FILE:
    # Add the series into the data frame 
    census_prep_df['block_fips'] = result_arry.values
    census_prep_df['county_fips'] = county_array.values

In [224]:
if not USE_CENSUS_BACKUP_FILE:
    census_prep_df.head

### Choose to save the data to a file

In [225]:
## Save the data to a file 
# if not USE_CENSUS_BACKUP_FILE:
#    census_prep_df.to_csv('1.3-rec-connecting-fips-data.csv')

# Add Ecosystem Data to the Working Dataset

### Notes: The file 1.3-rec-connecting-fips-data.csv is the file I passed to Nathan for Ecosystem Processing.  He then returned to me a dataset with the ecosystem data added as columns. The next section will proceed to to add in the ecosystme data  

## Load in Ecosystem data for the CBC Circles 

In [226]:
eco_data = pd.read_csv(PATH_TO_CBC_ECO_DATA)

In [227]:
eco_data.shape

(4531, 15)

### Notes On Definitions 
Ecosys - The numberic code for an ecosystme provided by USGS https://www.arcgis.com/home/item.html?id=8e8015c1e60b431fb191b5ed0de97b33. Translates into the Usgsid_sys human readable value 
Usgsid_sys - Human Readable Ecosystem label 
Nlcd_code - The numberic code for an  National Land Cover Database code provided by USGS https://www.arcgis.com/home/item.html?id=8e8015c1e60b431fb191b5ed0de97b33. Translates into the Nlcd human readable value 
Nlcd - Human Readable Ecosystem label of National Land Cover Code

In [228]:
# Take the Columns we Need
eco_data = eco_data[["lat","lon","Ecosys", "Usgsid_sys", "Nlcd_code", "Nlcd"]]

In [229]:
print(eco_data.shape)
eco_data.head(20)

(4531, 6)


,lat,lon,Ecosys,Usgsid_sys,Nlcd_code,Nlcd
0,36.616700,-121.916700,66.0,66_California Coastal Live Oak Woodland and Sa...,3.0,Steppe/Savanna
1,38.175251,-104.519575,274.0,274_Western Great Plains Shortgrass Prairie,4.0,Herbaceous
2,41.671800,-72.949500,300.0,300_Appalachian (Hemlock)-Northern Hardwood Fo...,1.0,Forest and Woodland
3,41.116700,-73.400000,487.0,487_Northern Atlantic Coastal Plain Pitch Pine...,1.0,Forest and Woodland
4,41.883300,-88.066700,254.0,254_North-Central Interior Beech-Maple Forest,1.0,Forest and Woodland
5,29.833300,-91.550000,337.0,337_West Gulf Coastal Plain Nonriverine Wet Ha...,5.0,Woody Wetland
6,42.383300,-71.166700,301.0,301_Northeastern Interior Dry-Mesic Oak Forest,1.0,Forest and Woodland
7,42.397309,-71.095501,324.0,324_Laurentian-Acadian Northern Hardwoods Forest,1.0,Forest and Woodland
8,42.450000,-71.133300,324.0,324_Laurentian-Acadian Northern Hardwoods Forest,1.0,Forest and Woodland
9,40.050000,-91.500000,250.0,250_North-Central Interior Oak Savanna,3.0,Steppe/Savanna


In [230]:
# Create a temporary key to merge on
eco_data['temp_key_str'] = round(eco_data['lat'],3).astype(str) + round(eco_data['lon'],3).astype(str)


In [231]:
# Some of the Ecosystem data is blank, lets put the lat lons that did NOT return ecosystem data into a new df 
na_eco_data = eco_data.loc[(eco_data["Ecosys"].isna() | (eco_data["Ecosys"] == 602.0))]

print(na_eco_data.shape)
na_eco_data

(181, 7)


,lat,lon,Ecosys,Usgsid_sys,Nlcd_code,Nlcd,temp_key_str
30,40.866700,-73.433300,602.0,"602_Unknown - pixel count <= 20,000",0.0,NaN,40.867-73.433
54,43.433300,-82.550000,602.0,"602_Unknown - pixel count <= 20,000",0.0,NaN,43.433-82.55
173,43.050000,-74.350000,602.0,"602_Unknown - pixel count <= 20,000",0.0,NaN,43.05-74.35
306,40.600000,-74.100000,602.0,"602_Unknown - pixel count <= 20,000",0.0,NaN,40.6-74.1
440,41.983300,-76.516700,602.0,"602_Unknown - pixel count <= 20,000",0.0,NaN,41.983-76.517
652,37.750000,-122.433300,602.0,"602_Unknown - pixel count <= 20,000",0.0,NaN,37.75-122.433
732,40.766700,-73.750000,602.0,"602_Unknown - pixel count <= 20,000",0.0,NaN,40.767-73.75
959,40.800000,-73.733300,602.0,"602_Unknown - pixel count <= 20,000",0.0,NaN,40.8-73.733
967,41.166700,-71.566700,602.0,"602_Unknown - pixel count <= 20,000",0.0,NaN,41.167-71.567
1012,41.766700,-70.100000,602.0,"602_Unknown - pixel count <= 20,000",0.0,NaN,41.767-70.1


In [232]:
# Limit eco data to only rows that contain data
eco_data = eco_data.loc[(eco_data["Ecosys"].notnull() & (eco_data["Ecosys"] != 602.0))]

print(eco_data.shape)
eco_data

(4350, 7)


,lat,lon,Ecosys,Usgsid_sys,Nlcd_code,Nlcd,temp_key_str
0,36.616700,-121.916700,66.0,66_California Coastal Live Oak Woodland and Sa...,3.0,Steppe/Savanna,36.617-121.917
1,38.175251,-104.519575,274.0,274_Western Great Plains Shortgrass Prairie,4.0,Herbaceous,38.175-104.52
2,41.671800,-72.949500,300.0,300_Appalachian (Hemlock)-Northern Hardwood Fo...,1.0,Forest and Woodland,41.672-72.95
3,41.116700,-73.400000,487.0,487_Northern Atlantic Coastal Plain Pitch Pine...,1.0,Forest and Woodland,41.117-73.4
4,41.883300,-88.066700,254.0,254_North-Central Interior Beech-Maple Forest,1.0,Forest and Woodland,41.883-88.067
...,...,...,...,...,...,...,...
4526,41.088856,-96.234197,253.0,253_North-Central Interior Floodplain,7.0,Mixed Upland and Wetland,41.089-96.234
4527,41.879000,-119.058500,190.0,190_Inter-Mountain Basins Big Sagebrush Steppe,3.0,Steppe/Savanna,41.879-119.058
4528,42.133971,-75.081353,322.0,322_Acadian-Appalachian Montane Spruce-Fir Forest,1.0,Forest and Woodland,42.134-75.081
4529,38.684400,-78.127700,301.0,301_Northeastern Interior Dry-Mesic Oak Forest,1.0,Forest and Woodland,38.684-78.128


In [233]:
# Investigate the duplicates 
eco_data['temp_key_dup'] = eco_data.duplicated(subset=['temp_key_str'], keep=False)
print("Are there any duplicate keys? : " + str(any(eco_data['temp_key_dup'])))

eco_data.sort_values(by=['temp_key_dup', 'temp_key_str'], inplace=True, ascending = False)

eco_data.head(100)



Are there any duplicate keys? : True


,lat,lon,Ecosys,Usgsid_sys,Nlcd_code,Nlcd,temp_key_str,temp_key_dup
346,41.700000,-70.300000,487.0,487_Northern Atlantic Coastal Plain Pitch Pine...,1.0,Forest and Woodland,41.7-70.3,True
3349,41.700378,-70.300172,487.0,487_Northern Atlantic Coastal Plain Pitch Pine...,1.0,Forest and Woodland,41.7-70.3,True
145,49.000000,-122.750000,130.0,130_North Pacific Oak Woodland,1.0,Forest and Woodland,49.0-122.75,False
1890,48.983300,-97.266700,251.0,251_North-Central Interior Maple-Basswood Forest,1.0,Forest and Woodland,48.983-97.267,False
1462,48.916700,-122.333300,528.0,528_North Pacific Maritime Mesic-Wet Douglas-f...,1.0,Forest and Woodland,48.917-122.333,False
2882,48.883300,-100.083300,260.0,"260_Eastern Great Plains Wet Meadow, Prairie, ...",6.0,Herbaceous Wetland,48.883-100.083,False
3897,48.876853,-115.051908,168.0,168_Northern Rocky Mountain Mesic Montane Mixe...,1.0,Forest and Woodland,48.877-115.052,False
3730,48.869758,-95.768551,261.0,261_Northern Tallgrass Prairie,4.0,Herbaceous,48.87-95.769,False
1224,48.866700,-101.516700,261.0,261_Northern Tallgrass Prairie,4.0,Herbaceous,48.867-101.517,False
3797,48.833238,-97.883471,259.0,259_Eastern Great Plains Tallgrass Aspen Parkland,1.0,Forest and Woodland,48.833-97.883,False


In [234]:
# Drop the duplicates 
eco_data.drop_duplicates(subset= ['temp_key_str'], inplace = True)
print(eco_data.shape)

eco_data['temp_key_dup'] = eco_data.duplicated(subset=['temp_key_str'], keep=False)
print("Are there any duplicate keys? : " + str(any(eco_data['temp_key_dup'])))

(4349, 8)
Are there any duplicate keys? : False


In [235]:
eco_data.sort_values(by=['temp_key_str']).head()


,lat,lon,Ecosys,Usgsid_sys,Nlcd_code,Nlcd,temp_key_str,temp_key_dup
1371,24.580846,-81.704636,423.0,423_South Florida Pine Rockland,1.0,Forest and Woodland,24.581-81.705,False
2032,24.627900,-82.872200,601.0,"601_Unknown - pixel count > 20,000",0.0,NaN,24.628-82.872,False
1878,24.666700,-81.350000,423.0,423_South Florida Pine Rockland,1.0,Forest and Woodland,24.667-81.35,False
2164,24.733300,-81.000000,423.0,423_South Florida Pine Rockland,1.0,Forest and Woodland,24.733-81.0,False
2320,24.866700,-80.766700,600.0,600_Water,0.0,NaN,24.867-80.767,False


## Now Load and Merge in the Station Eco Data
We wont need a temporary key for this file because the station id's are unique

In [236]:
station_eco_data = pd.read_csv(PATH_TO_NOAA_ECO_DATA)

In [237]:
station_eco_data.head()

,X,id,latitude,longitude,RASTERVALU,Red,Green,Blue,Opacity,Ecosys,Usgsid_sys,Nlcd_code,Nlcd
0,0,USC00500252,51.3833,179.2833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,USW00014607,46.8706,-68.0172,324.0,0.504556,0.623333,0.369333,1.0,324.0,324_Laurentian-Acadian Northern Hardwoods Forest,1.0,Forest and Woodland
2,9,USC00176937,46.6539,-68.0089,325.0,0.469988,0.594037,0.333426,1.0,325.0,325_Laurentian-Acadian Pine-Hemlock-Hardwood F...,1.0,Forest and Woodland
3,49,US1MEAR0015,46.6796,-68.0127,324.0,0.504556,0.623333,0.369333,1.0,324.0,324_Laurentian-Acadian Northern Hardwoods Forest,1.0,Forest and Woodland
4,56,USC00171833,45.6611,-67.8614,324.0,0.504556,0.623333,0.369333,1.0,324.0,324_Laurentian-Acadian Northern Hardwoods Forest,1.0,Forest and Woodland


In [238]:
station_eco_data.shape

(11652, 13)

## Merge in the FIPs census data, the CBC circle Ecosystem data, and the NOAA station data with the Station Matched Data 

In [239]:
# Load in the file of noaa matched cbc circles
full_working_df = pd.read_csv(PATH_TO_WORKING_DATA, compression = "gzip", sep = "\t")

/Users/rcdebaca/.pyenv/versions/funhacks371/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (65) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [240]:
full_working_df.head()

,circle_name,country_state,lat,lon,count_year,count_date,n_field_counters,n_feeder_counters,min_field_parties,max_field_parties,field_hours,feeder_hours,nocturnal_hours,field_distance,nocturnal_distance,distance_units,min_temp,max_temp,temp_unit,min_wind,max_wind,wind_unit,min_snow,max_snow,snow_unit,am_cloud,pm_cloud,field_distance_imperial,field_distance_metric,nocturnal_distance_imperial,nocturnal_distance_metric,min_snow_imperial,min_snow_metric,max_snow_metric,max_snow_imperial,min_temp_imperial,max_temp_imperial,min_temp_metric,max_temp_metric,min_wind_metric,max_wind_metric,min_wind_imperial,max_wind_imperial,ui,geohash_circle,circle_id,id,latitude,longitude,elevation,state,name,gsn_flag,hcn_crn_flag,wmoid,geohash_station,temp_min_value,temp_max_value,precipitation_value,temp_avg,snow,snwd,am_rain,pm_rain,am_snow,pm_snow,circle_elev
0,Hawai'i: Volcano N.P.,US-HI,19.433300,-155.283300,1955,1955-01-01,11.0,NaN,NaN,NaN,23.0,NaN,NaN,45.0,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.0,72.417123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.4333-155.2833_1955,8e3w,8e3wd3w,USC00511303,19.4297,-155.2561,1210.4,HI,HAWAII VOL NP HQ 54,,,NaN,8e3w,100.0,161.0,180.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,1228.18
1,Hawai'i: Volcano N.P.,US-HI,19.433300,-155.283300,1956,1955-12-31,11.0,NaN,NaN,NaN,32.0,NaN,NaN,104.0,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.0,167.364017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.4333-155.2833_1956,8e3w,8e3wd3w,USC00511303,19.4297,-155.2561,1210.4,HI,HAWAII VOL NP HQ 54,,,NaN,8e3w,117.0,189.0,290.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,1228.18
2,Hawai'i: Volcano N.P.,US-HI,19.433300,-155.283300,1968,1967-12-30,2.0,NaN,NaN,NaN,14.0,NaN,NaN,62.0,NaN,Miles,54.0,66.0,NaN,3.0,6.0,NaN,0.0,0.0,NaN,2.0,2.0,62.0,99.774702,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,12.222222,18.888889,4.827808,9.655616,1.8642,3.7284,19.4333-155.2833_1968,8e3w,8e3wd3w,USC00511033,19.4022,-155.2803,1109.5,HI,HALEMAUMAU 52,,,NaN,8e3w,NaN,NaN,NaN,NaN,0.0,0.0,3,2,3.0,3,1228.18
3,Hawai'i: Volcano N.P.,US-HI,19.433300,-155.283300,1968,1967-12-30,2.0,NaN,NaN,NaN,14.0,NaN,NaN,62.0,NaN,Miles,54.0,66.0,NaN,3.0,6.0,NaN,0.0,0.0,NaN,2.0,2.0,62.0,99.774702,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,12.222222,18.888889,4.827808,9.655616,1.8642,3.7284,19.4333-155.2833_1968,8e3w,8e3wd3w,USC00511303,19.4297,-155.2561,1210.4,HI,HAWAII VOL NP HQ 54,,,NaN,8e3w,100.0,189.0,0.0,NaN,0.0,0.0,3,2,3.0,3,1228.18
4,Hawai'i: Volcano,US-HI,19.516651,-155.299965,1973,1972-12-30,14.0,0.0,5.0,5.0,57.0,0.0,0.0,169.0,0.0,Miles,37.0,75.0,2.0,0.0,5.0,1.0,0.0,0.0,2.0,1.0,2.0,169.0,271.966527,0.0,0.0,0.0,0.0,0.0,0.0,37.0,75.0,2.777778,23.888889,0.000000,8.046347,0.0000,5.0000,19.516651-155.299965_1973,8e3x,8e3x40f,USC00519025,19.6581,-155.1325,320.0,HI,WAIAKEA SCD 88.2,,,NaN,8e3x,NaN,NaN,86.0,NaN,0.0,0.0,2,2,3.0,3,1551.44


In [241]:
full_working_df.shape

(127174, 67)

In [242]:
# create a Key to merge on
full_working_df['temp_key_str'] = round(full_working_df['lat'],3).astype(str) + round(full_working_df['lon'],3).astype(str)



In [243]:
full_working_df['temp_key_str'].head()

0    19.433-155.283
1    19.433-155.283
2    19.433-155.283
3    19.433-155.283
4      19.517-155.3
Name: temp_key_str, dtype: object

In [244]:
full_working_df['temp_key_str'].nunique()

3500

In [245]:
# Merge in the FIPS data with the full station data
full_working_df = pd.merge(full_working_df, census_prep_df[["temp_key_str", "block_fips", "county_fips"]], how="left", left_on="temp_key_str", right_on="temp_key_str")




In [246]:
full_working_df.shape

(127174, 70)

In [247]:
# Merge in the CBC Circle eco data 
full_working_df = pd.merge(full_working_df, eco_data[["temp_key_str","Ecosys", "Usgsid_sys", "Nlcd_code", "Nlcd"]], how="left", left_on= "temp_key_str", right_on = "temp_key_str")


In [248]:
full_working_df.shape

(127174, 74)

In [249]:
print(full_working_df['Ecosys'].value_counts())
print("The number of NAs:" + str(full_working_df['Ecosys'].isna().sum()))
print("The number of circles with %s:" % 'Ecosys' + str(full_working_df.shape[0] - full_working_df['Ecosys'].isna().sum()))



600.0    8672
254.0    8150
300.0    5728
301.0    3639
416.0    3308
324.0    3031
264.0    2615
439.0    2132
112.0    2079
274.0    2020
325.0    1868
250.0    1863
320.0    1799
251.0    1720
487.0    1684
265.0    1545
508.0    1541
272.0    1446
287.0    1430
68.0     1307
191.0    1295
79.0     1281
188.0    1217
478.0    1210
261.0    1197
61.0     1167
104.0    1158
409.0    1026
428.0    1012
288.0     975
408.0     943
276.0     908
221.0     903
148.0     893
130.0     889
480.0     875
76.0      873
10.0      858
412.0     835
302.0     821
262.0     806
154.0     766
69.0      742
499.0     732
292.0     715
438.0     699
601.0     699
147.0     683
49.0      682
589.0     663
67.0      643
469.0     629
207.0     612
128.0     612
180.0     599
278.0     592
224.0     584
71.0      577
172.0     561
239.0     557
83.0      554
297.0     554
582.0     547
11.0      542
538.0     540
574.0     539
165.0     520
185.0     514
322.0     500
417.0     491
72.0      478
227.0 

In [250]:
full_working_df.head(50)

,circle_name,country_state,lat,lon,count_year,count_date,n_field_counters,n_feeder_counters,min_field_parties,max_field_parties,field_hours,feeder_hours,nocturnal_hours,field_distance,nocturnal_distance,distance_units,min_temp,max_temp,temp_unit,min_wind,max_wind,wind_unit,min_snow,max_snow,snow_unit,am_cloud,pm_cloud,field_distance_imperial,field_distance_metric,nocturnal_distance_imperial,nocturnal_distance_metric,min_snow_imperial,min_snow_metric,max_snow_metric,max_snow_imperial,min_temp_imperial,max_temp_imperial,min_temp_metric,max_temp_metric,min_wind_metric,max_wind_metric,min_wind_imperial,max_wind_imperial,ui,geohash_circle,circle_id,id,latitude,longitude,elevation,state,name,gsn_flag,hcn_crn_flag,wmoid,geohash_station,temp_min_value,temp_max_value,precipitation_value,temp_avg,snow,snwd,am_rain,pm_rain,am_snow,pm_snow,circle_elev,temp_key_str,block_fips,county_fips,Ecosys,Usgsid_sys,Nlcd_code,Nlcd
0,Hawai'i: Volcano N.P.,US-HI,19.433300,-155.283300,1955,1955-01-01,11.0,NaN,NaN,NaN,23.00,NaN,NaN,45.0,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.0,72.417123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.4333-155.2833_1955,8e3w,8e3wd3w,USC00511303,19.4297,-155.2561,1210.4,HI,HAWAII VOL NP HQ 54,,,NaN,8e3w,100.0,161.0,180.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,1228.18,19.433-155.283,NaN,NaN,NaN,NaN,NaN,NaN
1,Hawai'i: Volcano N.P.,US-HI,19.433300,-155.283300,1956,1955-12-31,11.0,NaN,NaN,NaN,32.00,NaN,NaN,104.0,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.0,167.364017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.4333-155.2833_1956,8e3w,8e3wd3w,USC00511303,19.4297,-155.2561,1210.4,HI,HAWAII VOL NP HQ 54,,,NaN,8e3w,117.0,189.0,290.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,1228.18,19.433-155.283,NaN,NaN,NaN,NaN,NaN,NaN
2,Hawai'i: Volcano N.P.,US-HI,19.433300,-155.283300,1968,1967-12-30,2.0,NaN,NaN,NaN,14.00,NaN,NaN,62.0,NaN,Miles,54.0,66.0,NaN,3.0,6.0,NaN,0.0,0.0,NaN,2.0,2.0,62.0,99.774702,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,12.222222,18.888889,4.827808,9.655616,1.8642,3.7284,19.4333-155.2833_1968,8e3w,8e3wd3w,USC00511033,19.4022,-155.2803,1109.5,HI,HALEMAUMAU 52,,,NaN,8e3w,NaN,NaN,NaN,NaN,0.0,0.0,3,2,3.0,3,1228.18,19.433-155.283,NaN,NaN,NaN,NaN,NaN,NaN
3,Hawai'i: Volcano N.P.,US-HI,19.433300,-155.283300,1968,1967-12-30,2.0,NaN,NaN,NaN,14.00,NaN,NaN,62.0,NaN,Miles,54.0,66.0,NaN,3.0,6.0,NaN,0.0,0.0,NaN,2.0,2.0,62.0,99.774702,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,12.222222,18.888889,4.827808,9.655616,1.8642,3.7284,19.4333-155.2833_1968,8e3w,8e3wd3w,USC00511303,19.4297,-155.2561,1210.4,HI,HAWAII VOL NP HQ 54,,,NaN,8e3w,100.0,189.0,0.0,NaN,0.0,0.0,3,2,3.0,3,1228.18,19.433-155.283,NaN,NaN,NaN,NaN,NaN,NaN
4,Hawai'i: Volcano,US-HI,19.516651,-155.299965,1973,1972-12-30,14.0,0.0,5.0,5.0,57.00,0.0,0.0,169.0,0.0,Miles,37.0,75.0,2.0,0.0,5.0,1.0,0.0,0.0,2.0,1.0,2.0,169.0,271.966527,0.0,0.0,0.0,0.0,0.0,0.0,37.0,75.0,2.777778,23.888889,0.000000,8.046347,0.0000,5.0000,19.516651-155.299965_1973,8e3x,8e3x40f,USC00519025,19.6581,-155.1325,320.0,HI,WAIAKEA SCD 88.2,,,NaN,8e3x,NaN,NaN,86.0,NaN,0.0,0.0,2,2,3.0,3,1551.44,19.517-155.3,NaN,NaN,NaN,NaN,NaN,NaN
5,Hawai'i: Volcano,US-HI,19.516651,-155.299965,1973,1972-12-30,14.0,0.0,5.0,5.0,57.00,0.0,0.0,169.0,0.0,Miles,37.0,75.0,2.0,0.0,5.0,1.0,0.0,0.0,2.0,1.0,2.0,169.0,271.966527,0.0,0.0,0.0,0.0,0.0,0.0,37.0,75.0,2.777778,23.888889,0.000000,8.046347,0.0000,5.0000,19.516651-155.299965_1973,8e3x,8e3x40f,USC00516552,19.5486,-155.1100,466.3,HI,MTN VIEW 91,,,NaN,8e3x,144.0,244.0,18.0,NaN,0.0,0.0,2,2,3.0,3,1551.44,19.517-155.3,NaN,NaN,NaN,NaN,NaN,NaN
6,Hawai'i: Volcano,US-HI,19.516651,-155.299965,1973,1972-12-30,14.0,0.0,5.0,5.0,57.00,0.0,0.0,169.0,0.0,Miles,37.0,75.0,2.0,0.0,5.0,1.0,0.0,0.0,2.0,1.0,2.0,169.0,271.966527,0.0,0.0,0.0,0.0,0.0,0.0,37.0,75.0,2.777778,23.888889,0.000000,8.046347,0.0000,5.0000,19.516651-155.299965_1973,8e3x,8e3x40f,USC00511487,19.6833,-155.1667,487.7,HI,HILO COUNTRY CLUB 86,,,NaN,8e3x,NaN,NaN,3.0,NaN,0.0,0.0,2,2,3.0,3,1551.44,19.517-155.3,NaN,NaN,NaN,NaN,NaN,Na

In [251]:
# A good number of these guys did not get ecosystem data, let see if their lat lons are in the Na_eco_data set

# Get the list of temp keys with no Ecosys data from the merge
fw_nas_keys = full_working_df.loc[full_working_df["Ecosys"].isna(), "temp_key_str"]
print("The size of unmatech keys before dropping dups: " + str(fw_nas_keys.shape[0]))
fw_nas_keys.drop_duplicates(inplace = True)
print("The size of unmatech keys after dropping dups: " + str(fw_nas_keys.shape[0]))
print("This is the keys that were not able to be merged")

print("The size of the eco data set:" + str(na_eco_data.shape[0]))

The size of unmatech keys before dropping dups: 7802
The size of unmatech keys after dropping dups: 171
This is the keys that were not able to be merged
The size of the eco data set:181


In [252]:
# Now lets see how many of these are in the NA Eco Dataset 

# Get the Set Interaction of keys that are in the unmatched full working set keys that are 
# also in the list of ecosytem keys that have no data to offer. 
temp_keys_intersection = pd.Series(list(set(fw_nas_keys) & set(na_eco_data['temp_key_str'])))
len(temp_keys_intersection)

print("Out of %s keys in the Full Data Set that were unable to be matched, %s appeared also in the list of temperary keys in the eco data set that had no ecosystem data avilible (n = %s)" % (str(fw_nas_keys.shape[0]), str(len(temp_keys_intersection)), str(na_eco_data.shape[0]))) 



Out of 171 keys in the Full Data Set that were unable to be matched, 126 appeared also in the list of temperary keys in the eco data set that had no ecosystem data avilible (n = 181)


In [268]:
unmeged_keys_df = full_working_df.loc[ full_working_df['temp_key_str'].isin(list(fw_nas_keys)) & ~full_working_df['temp_key_str'].isin(list(temp_keys_intersection))]
unmeged_keys_df = unmeged_keys_df.drop_duplicates("temp_key_str")

unmeged_keys_df.shape

(45, 74)

In [270]:
unmeged_keys_df

,circle_name,country_state,lat,lon,count_year,count_date,n_field_counters,n_feeder_counters,min_field_parties,max_field_parties,field_hours,feeder_hours,nocturnal_hours,field_distance,nocturnal_distance,distance_units,min_temp,max_temp,temp_unit,min_wind,max_wind,wind_unit,min_snow,max_snow,snow_unit,am_cloud,pm_cloud,field_distance_imperial,field_distance_metric,nocturnal_distance_imperial,nocturnal_distance_metric,min_snow_imperial,min_snow_metric,max_snow_metric,max_snow_imperial,min_temp_imperial,max_temp_imperial,min_temp_metric,max_temp_metric,min_wind_metric,max_wind_metric,min_wind_imperial,max_wind_imperial,ui,geohash_circle,circle_id,id,latitude,longitude,elevation,state,name,gsn_flag,hcn_crn_flag,wmoid,geohash_station,temp_min_value,temp_max_value,precipitation_value,temp_avg,snow,snwd,am_rain,pm_rain,am_snow,pm_snow,circle_elev,temp_key_str,block_fips,county_fips,Ecosys,Usgsid_sys,Nlcd_code,Nlcd
6753,Clermont,us-fl,28.540414,-81.765054,2015,2015-01-02,37.0,1.0,1.0,10.0,111.10,2.00,9.0,568.65,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,6.0,568.65,915.111040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.540414000000002-81.765054_2015,djjf,djjf77s,US1FLOR0020,28.4952,-81.6498,40.8,FL,WINTER GARDEN 6.1 SW,,,NaN,djjf,NaN,NaN,0.0,NaN,0.0,NaN,3,3,3,3,32.94,28.54-81.765,NaN,NaN,NaN,NaN,NaN,NaN
95763,Northampton,us-ma,42.343900,-72.602800,1935,1934-12-23,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.3439-72.6028_1935,drs2,drs2guk,USC00193702,42.2000,-72.6000,29.9,MA,HOLYOKE,,,NaN,drs2,NaN,NaN,41.0,NaN,25.0,25.0,NaN,NaN,NaN,NaN,37.54,42.344-72.603,NaN,NaN,NaN,NaN,NaN,NaN
99852,Brodhead,us-wi,42.664372,-89.407110,2006,2005-12-14,6.0,NaN,5.0,5.0,25.75,NaN,1.5,292.50,10.00,Miles,26.0,30.0,2.0,10.0,20.0,1.0,4.00,8.00,2.0,2.0,2.0,292.50,470.711297,10.00,16.092694,4.00,10.160,20.320,8.00,26.0,30.0,-3.333333,-1.111111,16.092694,32.185388,10.0,20.0,42.664372-89.40711_2006,dp86,dp86tqz,USC00471078,42.6333,-89.3867,240.8,WI,BRODHEAD,,HCN,NaN,dp86,-56.0,-17.0,18.0,NaN,33.0,152.0,4,4,2,2,251.41,42.664-89.407,NaN,NaN,NaN,NaN,NaN,NaN
100269,Burlington,us-wi,42.711082,-88.203800,1992,1992-01-01,6.0,NaN,4.0,4.0,36.25,NaN,4.5,289.50,48.00,Miles,32.0,34.0,2.0,5.0,15.0,1.0,0.00,1.00,2.0,2.0,2.0,289.50,465.883489,48.00,77.244931,0.00,0.000,2.540,1.00,32.0,34.0,0.000000,1.111111,8.046347,24.139041,5.0,15.0,42.711082-88.2038_1992,dp96,dp96bz7,USC00478723,42.6903,-88.0336,222.5,WI,UNION GROVE,,,NaN,dp96,NaN,NaN,0.0,NaN,0.0,0.0,4,4,4,4,243.92,42.711-88.204,NaN,NaN,NaN,NaN,NaN,NaN
104072,Clyde,us-wi,43.055750,-90.112610,1970,1969-12-26,2.0,NaN,1.0,1.0,11.00,NaN,NaN,41.00,NaN,Miles,15.0,20.0,2.0,0.0,5.0,1.0,NaN,NaN,2.0,2.0,2.0,41.00,65.980045,NaN,NaN,NaN,NaN,NaN,NaN,15.0,20.0,-9.444444,-6.666667,0.000000,8.046347,0.0,5.0,43.05575-90.11261_1970,9zxu,9zxuvqj,USC00472173,42.9608,-90.1161,338.3,WI,DODGEVILLE,,,NaN,9zxu,-89.0,-28.0,0.0,NaN,0.0,279.0,4,4,4,4,386.13,43.056-90.113,NaN,NaN,NaN,NaN,NaN,NaN
105068,Oconomowoc,us-wi,43.135958,-88.499744,1955,1954-12-26,1.0,NaN,1.0,1.0,7.50,NaN,NaN,9.00,NaN,Miles,38.0,38.0,2.0,5.0,5.0,1.0,NaN,NaN,2.0,2.0,2.0,9.00,14.483425,NaN,NaN,NaN,NaN,NaN,NaN,38.0,38.0,3.333333,3.333333,8.046347,8.046347,5.0,5.0,43.135958-88.49974399999999_1955,dp9j,dp9j6hs,USC00476200,43.1003,-88.5036,260.9,WI,OCONOMOWOC,,,NaN,dp9j,11.0,33.0,69.0,NaN,0.0,25.0,2,2,4.0,4.0,268.25,43.136-88.5,NaN,NaN,NaN,NaN,NaN,NaN
106930,Retreat,us-wi,43.432658,-91.057627,2010,2009-12-28,5.0,1.0,3.0,3.0,15.25,NaN,NaN,130.00,NaN,Miles,25.0,31.0,2.0,12.0,12.0,1.0,5.00,5.00,2.0,6.0,1.0,130.00,209.205021,NaN,NaN,5.00,12.700,12.700,5.00,25.0,31.0,-3.888889,-0.555556,19.311233,19.311233,12.0,12.0,43.432658-91.057627_2010,9zxp,9zxppft,USC00473038,43.5706,-91.2294,194.8,WI,GENOA DAM 8,,,NaN,9zxp,-61.0,-28.0,0.0,NaN,0.0,127.0,4,4,4,4,321.49,43.433-91.058,NaN,NaN,NaN,NaN,NaN,NaN
107764,La Farge,us-wi,43.574344,-90.631371,1993,1993-01-02,6.0,6.0,

In [190]:
# Merge in the NOAA Station Eco data 
full_working_df = pd.merge(full_working_df, station_eco_data[["id","Ecosys", "Usgsid_sys", "Nlcd_code", "Nlcd"]], how="left", left_on= "id", right_on = "id", suffixes = ("_circle", "_station"))


In [191]:
full_working_df.shape

(127174, 78)

In [192]:
full_working_df['ui'].nunique()

65080

In [193]:
full_working_df.head()

,circle_name,country_state,lat,lon,count_year,count_date,n_field_counters,n_feeder_counters,min_field_parties,max_field_parties,field_hours,feeder_hours,nocturnal_hours,field_distance,nocturnal_distance,distance_units,min_temp,max_temp,temp_unit,min_wind,max_wind,wind_unit,min_snow,max_snow,snow_unit,am_cloud,pm_cloud,field_distance_imperial,field_distance_metric,nocturnal_distance_imperial,nocturnal_distance_metric,min_snow_imperial,min_snow_metric,max_snow_metric,max_snow_imperial,min_temp_imperial,max_temp_imperial,min_temp_metric,max_temp_metric,min_wind_metric,max_wind_metric,min_wind_imperial,max_wind_imperial,ui,geohash_circle,circle_id,id,latitude,longitude,elevation,state,name,gsn_flag,hcn_crn_flag,wmoid,geohash_station,temp_min_value,temp_max_value,precipitation_value,temp_avg,snow,snwd,am_rain,pm_rain,am_snow,pm_snow,circle_elev,temp_key_str,block_fips,county_fips,Ecosys_circle,Usgsid_sys_circle,Nlcd_code_circle,Nlcd_circle,Ecosys_station,Usgsid_sys_station,Nlcd_code_station,Nlcd_station
0,Hawai'i: Volcano N.P.,US-HI,19.433300,-155.283300,1955,1955-01-01,11.0,NaN,NaN,NaN,23.0,NaN,NaN,45.0,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.0,72.417123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.4333-155.2833_1955,8e3w,8e3wd3w,USC00511303,19.4297,-155.2561,1210.4,HI,HAWAII VOL NP HQ 54,,,NaN,8e3w,100.0,161.0,180.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,1228.18,19.433-155.283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Hawai'i: Volcano N.P.,US-HI,19.433300,-155.283300,1956,1955-12-31,11.0,NaN,NaN,NaN,32.0,NaN,NaN,104.0,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.0,167.364017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.4333-155.2833_1956,8e3w,8e3wd3w,USC00511303,19.4297,-155.2561,1210.4,HI,HAWAII VOL NP HQ 54,,,NaN,8e3w,117.0,189.0,290.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,1228.18,19.433-155.283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Hawai'i: Volcano N.P.,US-HI,19.433300,-155.283300,1968,1967-12-30,2.0,NaN,NaN,NaN,14.0,NaN,NaN,62.0,NaN,Miles,54.0,66.0,NaN,3.0,6.0,NaN,0.0,0.0,NaN,2.0,2.0,62.0,99.774702,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,12.222222,18.888889,4.827808,9.655616,1.8642,3.7284,19.4333-155.2833_1968,8e3w,8e3wd3w,USC00511033,19.4022,-155.2803,1109.5,HI,HALEMAUMAU 52,,,NaN,8e3w,NaN,NaN,NaN,NaN,0.0,0.0,3,2,3.0,3,1228.18,19.433-155.283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Hawai'i: Volcano N.P.,US-HI,19.433300,-155.283300,1968,1967-12-30,2.0,NaN,NaN,NaN,14.0,NaN,NaN,62.0,NaN,Miles,54.0,66.0,NaN,3.0,6.0,NaN,0.0,0.0,NaN,2.0,2.0,62.0,99.774702,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,12.222222,18.888889,4.827808,9.655616,1.8642,3.7284,19.4333-155.2833_1968,8e3w,8e3wd3w,USC00511303,19.4297,-155.2561,1210.4,HI,HAWAII VOL NP HQ 54,,,NaN,8e3w,100.0,189.0,0.0,NaN,0.0,0.0,3,2,3.0,3,1228.18,19.433-155.283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Hawai'i: Volcano,US-HI,19.516651,-155.299965,1973,1972-12-30,14.0,0.0,5.0,5.0,57.0,0.0,0.0,169.0,0.0,Miles,37.0,75.0,2.0,0.0,5.0,1.0,0.0,0.0,2.0,1.0,2.0,169.0,271.966527,0.0,0.0,0.0,0.0,0.0,0.0,37.0,75.0,2.777778,23.888889,0.000000,8.046347,0.0000,5.0000,19.516651-155.299965_1973,8e3x,8e3x40f,USC00519025,19.6581,-155.1325,320.0,HI,WAIAKEA SCD 88.2,,,NaN,8e3x,NaN,NaN,86.0,NaN,0.0,0.0,2,2,3.0,3,1551.44,19.517-155.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Now Lets Look at Cleaning Up the Ecosystem data

In [194]:
# check that the merge went through
print("The number of Nans is: %s" % full_working_df['Ecosys_circle'].isna().sum())
full_working_df['Ecosys_circle'].value_counts(dropna = False)

The number of Nans is: 7802


600.0    8672
254.0    8150
NaN      7802
300.0    5728
301.0    3639
416.0    3308
324.0    3031
264.0    2615
439.0    2132
112.0    2079
274.0    2020
325.0    1868
250.0    1863
320.0    1799
251.0    1720
487.0    1684
265.0    1545
508.0    1541
272.0    1446
287.0    1430
68.0     1307
191.0    1295
79.0     1281
188.0    1217
478.0    1210
261.0    1197
61.0     1167
104.0    1158
409.0    1026
428.0    1012
288.0     975
408.0     943
276.0     908
221.0     903
148.0     893
130.0     889
480.0     875
76.0      873
10.0      858
412.0     835
302.0     821
262.0     806
154.0     766
69.0      742
499.0     732
292.0     715
438.0     699
601.0     699
147.0     683
49.0      682
589.0     663
67.0      643
469.0     629
207.0     612
128.0     612
180.0     599
278.0     592
224.0     584
71.0      577
172.0     561
239.0     557
297.0     554
83.0      554
582.0     547
11.0      542
538.0     540
574.0     539
165.0     520
185.0     514
322.0     500
417.0     491
72.0  

In [195]:
# check that the merge went through
full_working_df['Usgsid_sys_circle'].value_counts(dropna = False)

600_Water                                                                          8672
254_North-Central Interior Beech-Maple Forest                                      8150
NaN                                                                                7802
300_Appalachian (Hemlock)-Northern Hardwood Forest                                 5728
301_Northeastern Interior Dry-Mesic Oak Forest                                     3639
416_Northern Atlantic Coastal Plain Pitch Pine Lowland                             3308
324_Laurentian-Acadian Northern Hardwoods Forest                                   3031
264_Central Tallgrass Prairie                                                      2615
439_Southern Piedmont Dry Oak-(Pine) Forest                                        2132
112_South-Central Interior Mesophytic Forest                                       2079
274_Western Great Plains Shortgrass Prairie                                        2020
325_Laurentian-Acadian Pine-Heml

In [196]:
# check that the merge went through
full_working_df['Nlcd_code_circle'].value_counts(dropna = False)

1.0    59025
4.0    15800
0.0     9389
3.0     8884
2.0     8760
NaN     7802
7.0     6076
5.0     5820
8.0     4265
6.0     1353
Name: Nlcd_code_circle, dtype: int64

In [197]:
# check that the merge went through
full_working_df['Nlcd_circle'].value_counts(dropna = False)

Forest and Woodland         59025
NaN                         17191
Herbaceous                  15800
Steppe/Savanna               8884
Shrubland                    8760
Mixed Upland and Wetland     6076
Woody Wetland                5820
Barren                       4265
Herbaceous Wetland           1353
Name: Nlcd_circle, dtype: int64

In [198]:
# The Ecosys_circle code of 601 , 601_Unknown - pixel count > 20,000, is essentally nan so let make it so 
print("The number of Nans before the cleaning is: %s" % full_working_df['Ecosys_circle'].isna().sum())
print("The number of Nans before the cleaning is: %s" % full_working_df['Usgsid_sys_circle'].isna().sum())

full_working_df['Ecosys_circle'].replace(601.0, np.nan, inplace = True)
full_working_df['Usgsid_sys_circle'].replace('601_Unknown - pixel count > 20,000', np.nan, inplace = True)
full_working_df['Usgsid_sys_circle'].replace('0_n/a', np.nan, inplace = True)

print("The number of Nans after the cleaning is: %s" % full_working_df['Ecosys_circle'].isna().sum())
print("The number of Nans after the cleaning is: %s" % full_working_df['Usgsid_sys_circle'].isna().sum())


The number of Nans before the cleaning is: 7802
The number of Nans before the cleaning is: 7802
The number of Nans after the cleaning is: 8501
The number of Nans after the cleaning is: 8519


## Prepare the data for Saving

In [199]:
# Drop the temportary key 
full_working_df = full_working_df.drop("temp_key_str",axis=1)

In [200]:
full_working_df.to_csv('../data/Cloud_data/1.3-rec-connecting-fips-ecosystem-data.txt', compression = "gzip", sep="\t", index=False)

In [201]:
full_working_df['Ecosys_circle'].value_counts()
print("The number of NAs:" + str(full_working_df['Ecosys_circle'].isna().sum()))
print("The number of circles with %s:" % 'Ecosys_circle' + str(full_working_df.shape[0] - full_working_df['Ecosys_circle'].isna().sum()))



The number of NAs:8501
The number of circles with Ecosys_circle:118673
